## Import libraries

In [3]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 8.2 MB/s eta 0:00:00


In [4]:
import snscrape.modules.twitter as sntwitter
import datetime as dt
import pandas as pd


## Scrape tweets

Scrape 365k positive (using ":)" as the query) and 365k negative (using ":(" as the query) tweets from year 2022. We will scrape 1000 tweets per day for each category.

In [ ]:
def scrape_tweet(search_term, start_date, end_date, num_tweets):
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    tweet_data = []
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(
            "{} since:{} until:{} lang:en exclude:retweets".format(
                search_term, start_date, end_date
            )
        ).get_items()
    ):
        if i >= num_tweets:
            break
        tweet_data.append([tweet.user.username, tweet.date, tweet.content])
    tweet_df = pd.DataFrame(tweet_data, columns=["username", "date", "tweet"])
    return tweet_df

In [ ]:
def daily_scrape_2022(search_term, num_daily):
    start_date = dt.datetime(2022, 1, 1)
    end_date = dt.datetime(2022, 1, 2)
    delta = dt.timedelta(days=1)
    df = pd.DataFrame()
    for n in range(365):
        temp_df = scrape_tweet(search_term, start_date, end_date, num_daily)
        df = pd.concat([df, temp_df])
        start_date += delta
        end_date += delta
    return df

In [ ]:
ori_neg_df = daily_scrape_2022(":(", 1000)


In [ ]:
ori_pos_df = daily_scrape_2022(":)", 1000)


## Filter scraped tweets

In [ ]:
def filter_include(df, term_list):
    temp_df = pd.DataFrame()
    for term in term_list:
        add_df = df[df["tweet"].str.contains(term, regex=False) == True]
        temp_df = pd.concat([temp_df, add_df]).drop_duplicates(ignore_index=True)
    return temp_df

In [ ]:
def filter_exclude(df, term_list):
    temp_df = df.copy()
    for term in term_list:
        temp_df = temp_df[temp_df["tweet"].str.contains(term, regex=False) == False]
    return temp_df

Filter negative tweet



In [ ]:
neg_df = filter_include(ori_neg_df, [":(", ":-("])
neg_df = filter_exclude(neg_df, [":)", ":D", ":-)"])
neg_df.shape

(358624, 3)

Filter positive tweet 

In [ ]:
pos_df = filter_include(ori_pos_df, [":)", ":D", ":-)"])
pos_df = filter_exclude(pos_df, [":(", ":-("])
pos_df.shape

(343477, 3)

## Remove emojis from tweets

Remove all emojis because we want our model to classify the tweet sentiment from the text instead of emojis

In [ ]:
def remove_term(df, term_list):
    temp_df = df.copy()
    for term in term_list:
        temp_df["tweet"] = temp_df["tweet"].str.replace(term, " ", regex=False)
    return temp_df

In [ ]:
neg_df = remove_term(neg_df, [":(", ":-("])

In [ ]:
pos_df = remove_term(pos_df, [":)", ":D", ":-)"])

## Label tweets and merge them into a dataframe

In [ ]:
neg_df["sentiment"] = "Negative"
pos_df["sentiment"] = "Positive"
df = pd.concat([neg_df, pos_df]).reset_index(drop=True)

In [ ]:
df.to_csv("/content/drive/MyDrive/dataset/labeled_tweets.csv", index=False)